Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

In [19]:
import azureml.core
from azureml.core import Workspace, Experiment, Run, Environment
from azureml.core import ScriptRunConfig
from azureml.core.compute import AmlCompute,ComputeTarget


In [20]:
ws = Workspace.from_config()
#TODO: load variables from config file

exp_name = ""
compute_name = "cpucompute"
env_name = ""

project_dir = '../src'
script_name = 'train.py'
model_name = ""
model_output_dir = 'outputs/'
dataset_name = ""

In [21]:
experiment = Experiment(workspace=ws, name=exp_name)

In [22]:
if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]

In [23]:
# env = Environment.get(workspace=ws, name=env_name)
# env = Environment.load_from_directory('../configuration/environments/environment_training')
env = Environment('trainingEnv')
for pip_package in ["joblib","scikit-learn","pandas","azureml-sdk"]:
    env.python.conda_dependencies.add_pip_package(pip_package)

In [24]:
src = ScriptRunConfig(source_directory=project_dir,
                      script=script_name,
                      compute_target=compute_target,
                      environment=env,
                      arguments=[
                          '--dataset-name',dataset_name,
                          '--model-name',model_name,
                          '--output-dir',model_output_dir
                          ]
                      )


In [25]:
run = experiment.submit(config=src)
print(run.get_portal_url())


https://ml.azure.com/experiments/train-remote/runs/train-remote_1612952327_993b0ec1?wsid=/subscriptions/ed835daf-8df9-4922-a58f-74a80da0b5c0/resourcegroups/mlops-uc-sales/workspaces/mlouc1daml


In [35]:
print(run.get_status())
#run.wait_for_completion(show_output=True)

Completed


In [34]:
if run.get_status() == 'Completed':
    model = run.register_model(
        model_name=model_name,
        model_path=f'{model_output_dir}{model_name}'
        )